In [ ]:
"""
Getting Started.

This notebook shows you the basic steps for loading data from the eBird API
and displaying checklists and observations. It assumes you have completed
the steps in the "Getting Started" section of the README, and you have an
eBird API key.

"""
from django.conf import settings
from django.template.loader import render_to_string

from faker import Faker
from IPython.display import display, HTML

from ebird.api.requests import get_visits
from ebird.api.data.loaders import APILoader
from ebird.api.data.models import Checklist, Observation

# Randomly choose a country to load checklists for.
region = Faker().country_code()

In [ ]:
# Load the most recent checklists.
loader = APILoader(settings.EBIRD_API_KEY, locales=settings.EBIRD_API_LOCALES)

# Normally you use the APILoader for this, however if the country
# chosen at random is a big birding nation, then we're going to
# overload the eBird servers with requests. This way is safer.
# Get the visits from the API, the default number is 10.
visits = get_visits(settings.EBIRD_API_KEY, region)

for visit in visits:
    loader.add_location(visit["loc"])
    loader.load_checklist(visit["subId"])

In [ ]:
# Show the 10 most recent checklists.
checklists = Checklist.objects.filter(location__country__code=region)

table = render_to_string("ebird-api-basics/getting-started/checklists.html", {"checklists": checklists})
display(HTML(table))

In [ ]:
# Show the highest count for each species seen
# Rather than build a complex query to group the observations by species
# and calculate the highest count, we cheat by using the {% ifchanged %}
# tag in the template to display the observation each time the species
# changes. Sorting by count descending means this record is the one where
# the highest count was recorded.
observations = Observation.objects.filter(location__country__code=region).order_by("species__taxon_order", "-count")

table = render_to_string("ebird-api-basics/getting-started/highest-counts.html", {"observations": observations})
display(HTML(table))